# Hugging Face Pipeline Behind The Scene

Coded and shared by Divya Patel, Microsoft.

In [2]:
from transformers import pipeline

In [3]:
# Text classification
classifier = pipeline("sentiment-analysis")
classifier("We are very happy to show you the 🤗 Transformers library.")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that aut

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

[{'label': 'POSITIVE', 'score': 0.9997795224189758}]

## What's happening behind the scene of Hugging Face pipeline?

1. **Load the pre-trained model**: First, you need to load the pre-trained model that is suitable for the specific NLP task you want to perform. Hugging Face provides a wide range of pre-trained models for different tasks and languages.

2. **Tokenization**: The input text is tokenized, which means it is split into individual words or subwords. Tokenization is an important step in NLP as it helps the model understand the structure and meaning of the text.

3. **Model Inference**: The tokenized input is passed through the pre-trained model, which generates predictions or outputs based on the specific task. For example, in text classification, the model predicts the class or category of the input text.

4. **Post-processing**: The output generated by the model is post-processed to obtain the final result. This may involve converting the model's output into a human-readable format or applying additional logic to extract relevant information.

5. **Return the result**: Finally, the processed result is returned to the user. This could be the predicted class label, extracted entities, or any other relevant information based on the specific task.

Note: The exact implementation details may vary depending on the specific pipeline and task being performed.

## Step 1: Load the pre-trained model

In [4]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased-finetuned-sst-2-english")

In [5]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

## Step 2: Tokenization

In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased-finetuned-sst-2-english")

In [7]:
# Tokenized the input
inputs = tokenizer("We are very happy to show you the 🤗 Transformers library.", return_tensors="pt")
print(inputs)

{'input_ids': tensor([[  101,  2057,  2024,  2200,  3407,  2000,  2265,  2017,  1996,   100,
         19081,  3075,  1012,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


## Step 3: Model Inference

In [8]:
outputs = model(**inputs)
print(outputs)

SequenceClassifierOutput(loss=None, logits=tensor([[-4.0833,  4.3364]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


## Step 4: Post Processing

In [9]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

In [10]:
predicted_class_idx = outputs.logits.argmax().item()
print(f"label: {model.config.id2label[predicted_class_idx]}, score: {outputs.logits.softmax(dim=1)[0, predicted_class_idx]}")

label: POSITIVE, score: 0.9997795224189758
